In [6]:
import pandas as pd
import pgmpy
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from matplotlib.pyplot import hist

ModuleNotFoundError: No module named 'torch'

In [ ]:
data = pd.read_csv("./london_merged.csv")
# data["date"] = pd.to_datetime(data["timestamp"]).dt.date.astype(str)
data["time"] = pd.to_datetime(data["timestamp"]).dt.time.astype(str).str.slice(0,2).astype(int)
data = data.drop("timestamp", axis = 1)
data = data.drop("t2", axis = 1)
data = data.drop("hum", axis = 1)
data = data.drop("is_holiday", axis = 1)

data = data[data['cnt'] < 5000]
data = data[data['time'] > 6]
data['is_weekend'] = data['is_weekend'].map({1: "yes", 0: "no"})
data['weather'] = data['weather_code'].map({
    1: "clear", 
    2: "clear", 
    3: "clear", 
    4: "clear", 
    7: "rain", 
    10: "rain",
    26: "rain",
    94: "rain"
})
data['season'] = data['season'].map({
    0: "spring",
    1: "summer",
    2: "fall",
    3: "winter"
})
data["wind"] = data['wind_speed']
data = data.drop('wind_speed', axis = 1)
data["temperature"] = data["t1"]
data = data.drop("t1", axis = 1)
data = data.drop("weather_code", axis = 1)

# hist(data.time, weights=data.cnt);
data.head()


,cnt,is_weekend,season,time,weather,wind,temperature
7,75,yes,winter,7,clear,7.0,1.0
8,131,yes,winter,8,clear,8.0,1.5
9,301,yes,winter,9,clear,9.0,2.0
10,528,yes,winter,10,clear,12.0,3.0
11,727,yes,winter,11,clear,12.0,2.0


In [ ]:

data['temperature'] = pd.cut(data['temperature'], bins=4, labels=[0, 1, 2, 3])
# data['t2'] = pd.cut(data['t2'], bins=4, labels=[0, 1, 2, 3])
data['time'] = pd.cut(data['time'], bins=2, labels=["7-14", "15-23"])
# data['hum'] = pd.cut(data['hum'], bins=4, labels=[0, 1, 2, 3])
data['wind'] = pd.cut(data['wind'], bins=2, labels=["yes", "no"])
data['cnt'] = pd.qcut(data['cnt'], q=8, labels=[0, 1, 2, 3, 4, 5, 6, 7])

In [ ]:
data.head()

,cnt,is_weekend,season,time,weather,wind,temperature
7,0,yes,winter,7-14,clear,yes,0
8,0,yes,winter,7-14,clear,yes,0
9,0,yes,winter,7-14,clear,yes,0
10,1,yes,winter,7-14,clear,yes,0
11,1,yes,winter,7-14,clear,yes,0


In [ ]:
network = BayesianNetwork()
network.add_nodes_from(["cnt", "temperature", "wind", "weather", "is_weekend", "season", "time"])

network.add_edge("is_weekend", "cnt")
network.add_edge("season", "weather")
network.add_edge("time", "temperature")
network.add_edge("time", "cnt")
network.add_edge("weather", "temperature")
network.add_edge("weather", "wind")
network.add_edge("wind", "cnt")
network.add_edge("temperature", "cnt")

In [ ]:
def createCPD(name, parents):
    cpd = []
    for v in data[name].unique():
        for p in parents:
            row = []
            for vp in data[p].unique():
                row.append(data[(data[p] == vp) & (data[name] == v)].shape[0] / data[data[p] == vp].shape[0])

        cpd.append(row)

createCPD("weather", ["season"])

NameError: name 'data' is not defined

In [ ]:
n_rows = data.shape[0]
prob_not_weekend = data[data['is_weekend'] == "no"].shape[0] / n_rows
cpd_is_we = TabularCPD('is_weekend', 2, [
    [prob_not_weekend], 
    [1 - prob_not_weekend]
])
print(cpd_is_we)

cpd_season = TabularCPD('season', 4, [
    [data[data['season'] == "spring"].shape[0] / n_rows],
    [data[data['season'] == "summer"].shape[0] / n_rows],
    [data[data['season'] == "fall"].shape[0] / n_rows],
    [data[data['season'] == "winter"].shape[0] / n_rows]
])
print(cpd_season)

cpd_time = TabularCPD('time', 2, [
    [data[data['time'] == "7-14"].shape[0] / n_rows],
    [data[data['time'] == "15-23"].shape[0] / n_rows]
])
print(cpd_time)

cpd_weather = TabularCPD('weather', 2, [
    [
        data[(data['weather'] == "clear") & (data["season"] == "spring")].shape[0] / data[data["season"] == "spring"].shape[0], 
        data[(data['weather'] == "clear") & (data["season"] == "summer")].shape[0] / data[data["season"] == "summer"].shape[0], 
        data[(data['weather'] == "clear") & (data["season"] == "fall")].shape[0] / data[data["season"] == "fall"].shape[0], 
        data[(data['weather'] == "clear") & (data["season"] == "winter")].shape[0] / data[data["season"] == "winter"].shape[0]
    ],
    [
        data[(data['weather'] == "rain") & (data["season"] == "spring")].shape[0] / data[data["season"] == "spring"].shape[0], 
        data[(data['weather'] == "rain") & (data["season"] == "summer")].shape[0] / data[data["season"] == "summer"].shape[0], 
        data[(data['weather'] == "rain") & (data["season"] == "fall")].shape[0] / data[data["season"] == "fall"].shape[0], 
        data[(data['weather'] == "rain") & (data["season"] == "winter")].shape[0] / data[data["season"] == "winter"].shape[0]
    ],
], evidence = ["season"], evidence_card = [4])
print(cpd_weather)

cpd_wind = TabularCPD('wind', 2, [
    [
        data[(data['wind'] == "no") & (data["weather"] == "clear")].shape[0] / data[data["weather"] == "clear"].shape[0], 
        data[(data['wind'] == "no") & (data["weather"] == "rain")].shape[0] / data[data["weather"] == "rain"].shape[0]
    ],
    [
        data[(data['wind'] == "yes") & (data["weather"] == "clear")].shape[0] / data[data["weather"] == "clear"].shape[0], 
        data[(data['wind'] == "yes") & (data["weather"] == "rain")].shape[0] / data[data["weather"] == "rain"].shape[0]
    ],
], evidence = ["weather"], evidence_card = [2])
print(cpd_wind)

cpd_temperature = TabularCPD('temperature', 4, [
    [
        data[(data['temperature'] == 0) & (data["weather"] == "clear") & (data['time'] == "7-14")].shape[0] / data[(data["weather"] == "clear") & (data['time'] == "7-14")].shape[0], 
        data[(data['temperature'] == 0) & (data["weather"] == "clear") & (data['time'] == "15-23")].shape[0] / data[(data["weather"] == "clear") & (data['time'] == "15-23")].shape[0],
        data[(data['temperature'] == 0) & (data["weather"] == "rain") & (data['time'] == "7-14")].shape[0] / data[(data["weather"] == "rain") & (data['time'] == "7-14")].shape[0], 
        data[(data['temperature'] == 0) & (data["weather"] == "rain") & (data['time'] == "15-23")].shape[0] / data[(data["weather"] == "rain") & (data['time'] == "15-23")].shape[0]
    ],
    [
        data[(data['temperature'] == 1) & (data["weather"] == "clear") & (data['time'] == "7-14")].shape[0] / data[(data["weather"] == "clear") & (data['time'] == "7-14")].shape[0], 
        data[(data['temperature'] == 1) & (data["weather"] == "clear") & (data['time'] == "15-23")].shape[0] / data[(data["weather"] == "clear") & (data['time'] == "15-23")].shape[0],
        data[(data['temperature'] == 1) & (data["weather"] == "rain") & (data['time'] == "7-14")].shape[0] / data[(data["weather"] == "rain") & (data['time'] == "7-14")].shape[0], 
        data[(data['temperature'] == 1) & (data["weather"] == "rain") & (data['time'] == "15-23")].shape[0] / data[(data["weather"] == "rain") & (data['time'] == "15-23")].shape[0]
    ],
    [
        data[(data['temperature'] == 2) & (data["weather"] == "clear") & (data['time'] == "7-14")].shape[0] / data[(data["weather"] == "clear") & (data['time'] == "7-14")].shape[0], 
        data[(data['temperature'] == 2) & (data["weather"] == "clear") & (data['time'] == "15-23")].shape[0] / data[(data["weather"] == "clear") & (data['time'] == "15-23")].shape[0],
        data[(data['temperature'] == 2) & (data["weather"] == "rain") & (data['time'] == "7-14")].shape[0] / data[(data["weather"] == "rain") & (data['time'] == "7-14")].shape[0], 
        data[(data['temperature'] == 2) & (data["weather"] == "rain") & (data['time'] == "15-23")].shape[0] / data[(data["weather"] == "rain") & (data['time'] == "15-23")].shape[0]
    ],
    [
        data[(data['temperature'] == 3) & (data["weather"] == "clear") & (data['time'] == "7-14")].shape[0] / data[(data["weather"] == "clear") & (data['time'] == "7-14")].shape[0], 
        data[(data['temperature'] == 3) & (data["weather"] == "clear") & (data['time'] == "15-23")].shape[0] / data[(data["weather"] == "clear") & (data['time'] == "15-23")].shape[0],
        data[(data['temperature'] == 3) & (data["weather"] == "rain") & (data['time'] == "7-14")].shape[0] / data[(data["weather"] == "rain") & (data['time'] == "7-14")].shape[0], 
        data[(data['temperature'] == 3) & (data["weather"] == "rain") & (data['time'] == "15-23")].shape[0] / data[(data["weather"] == "rain") & (data['time'] == "15-23")].shape[0]
    ],
], evidence = ["weather", "time"], evidence_card = [2, 2])
print(cpd_temperature)

+---------------+----------+
| is_weekend(0) | 0.713335 |
+---------------+----------+
| is_weekend(1) | 0.286665 |
+---------------+----------+
+-----------+----------+
| season(0) | 0.252902 |
+-----------+----------+
| season(1) | 0.250791 |
+-----------+----------+
| season(2) | 0.246896 |
+-----------+----------+
| season(3) | 0.249412 |
+-----------+----------+
+---------+----------+
| time(0) | 0.529908 |
+---------+----------+
| time(1) | 0.470092 |
+---------+----------+
+------------+---------------------+-----+---------------------+
| season     | season(0)           | ... | season(3)           |
+------------+---------------------+-----+---------------------+
| weather(0) | 0.8825417201540436  | ... | 0.8561666124308493  |
+------------+---------------------+-----+---------------------+
| weather(1) | 0.11745827984595636 | ... | 0.14383338756915068 |
+------------+---------------------+-----+---------------------+
+---------+---------------------+---------------------+
| we

In [ ]:
data[(data['weather'] == "clear") & (data["season"] == "spring")].shape[0] / data[data["season"] == "spring"].shape[0]

0.8825417201540436

In [ ]:
from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import BicScore
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator

hc = HillClimbSearch(data)
best_model = hc.estimate()

b_model = BayesianNetwork(best_model.edges())
b_model.fit(data, estimator = MaximumLikelihoodEstimator)
for cpd in b_model.get_cpds():
    print(cpd)

  0%|          | 23/1000000 [00:03<37:50:18,  7.34it/s]


+------------+-----+-----------------+
| is_holiday | ... | is_holiday(1.0) |
+------------+-----+-----------------+
| is_weekend | ... | is_weekend(1.0) |
+------------+-----+-----------------+
| t1         | ... | t1(3)           |
+------------+-----+-----------------+
| time       | ... | time(5)         |
+------------+-----+-----------------+
| cnt(0)     | ... | 0.125           |
+------------+-----+-----------------+
| cnt(1)     | ... | 0.125           |
+------------+-----+-----------------+
| cnt(2)     | ... | 0.125           |
+------------+-----+-----------------+
| cnt(3)     | ... | 0.125           |
+------------+-----+-----------------+
| cnt(4)     | ... | 0.125           |
+------------+-----+-----------------+
| cnt(5)     | ... | 0.125           |
+------------+-----+-----------------+
| cnt(6)     | ... | 0.125           |
+------------+-----+-----------------+
| cnt(7)     | ... | 0.125           |
+------------+-----+-----------------+
+--------+---------------